In [3]:
import pandas as pd

df = pd.read_csv('/content/HR Analytics.csv')


**1. Remove Non-Informative Columns These columns have no variance:**

EmployeeCount → Always 1

Over18 → Always 'Y'

StandardHours → Always 80

In [4]:
df.drop(columns=['EmployeeCount', 'Over18', 'StandardHours'], inplace=True)

**2. Rename Columns for Readability**

In [6]:
df.rename(columns={
    'DailyRate': 'Daily Rate',
    'DistanceFromHome': 'Distance From Home',
    'EducationField': 'Education Field',
    'EnvironmentSatisfaction': 'Environment Satisfaction',
    'HourlyRate': 'Hourly Rate',
    'JobInvolvement': 'Job Involvement',
    'JobLevel': 'Job Level',
    'JobSatisfaction': 'Job Satisfaction',
    'MaritalStatus': 'Marital Status',
    'MonthlyIncome': 'Monthly Income',
    'MonthlyRate': 'Monthly Rate',
    'NumCompaniesWorked': 'Num Companies Worked',
    'OverTime': 'Overtime',
    'PercentSalaryHike': 'Percent Salary Hike',
    'PerformanceRating': 'Performance Rating',
    'RelationshipSatisfaction': 'Relationship Satisfaction',
    'StockOptionLevel': 'Stock Option Level',
    'TotalWorkingYears': 'Total Working Years',
    'TrainingTimesLastYear': 'Training Times Last Year',
    'WorkLifeBalance': 'Work Life Balance',
    'YearsAtCompany': 'Years At Company',
    'YearsInCurrentRole': 'Years In Current Role',
    'YearsSinceLastPromotion': 'Years Since Last Promotion',
    'YearsWithCurrManager': 'Years With Current Manager'
}, inplace=True)


**3. Convert Ordinal Encoded Columns to Meaningful Labels**

These columns have rating scales (1–4 or 1–5). We can make them readable:

rating_map_4 = **{1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}**

rating_map_5 = **{1: 'Below College', 2: 'College', 3: 'Bachelor', 4: 'Master', 5: 'Doctor'}**

In [8]:
rating_map_4 = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
rating_map_5 = {1: 'Below College', 2: 'College', 3: 'Bachelor', 4: 'Master', 5: 'Doctor'}

df['Environment Satisfaction'] = df['Environment Satisfaction'].map(rating_map_4)
df['Job Involvement'] = df['Job Involvement'].map(rating_map_4)
df['Job Satisfaction'] = df['Job Satisfaction'].map(rating_map_4)
df['Relationship Satisfaction'] = df['Relationship Satisfaction'].map(rating_map_4)
df['Work Life Balance'] = df['Work Life Balance'].map(rating_map_4)
df['Education'] = df['Education'].map(rating_map_5)


**4. Format Categorical Values**

In [10]:
df['BusinessTravel'] = df['BusinessTravel'].replace({
    'Travel_Rarely': 'Travel Rarely',
    'Travel_Frequently': 'Travel Frequently',
    'Non-Travel': 'No Travel'
})
df['Overtime'] = df['Overtime'].map({'Yes': 'Overtime', 'No': 'No Overtime'})


**5. Data Type Optimization**

In [12]:
# Convert to category type where applicable
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols] = df[categorical_cols].astype('category')


**6. Handle Rare Categories**

Some categories (e.g., Human Resources department, only 4%) can be grouped under "Other" for certain visualizations, but keep original for completeness.

In [13]:
df['Department'] = df['Department'].apply(lambda x: x if df['Department'].value_counts()[x] > 50 else 'Other')


In [14]:
print(df.head())

   Age Attrition     BusinessTravel  Daily Rate              Department  \
0   41       Yes      Travel Rarely        1102                   Sales   
1   49        No  Travel Frequently         279  Research & Development   
2   37       Yes      Travel Rarely        1373  Research & Development   
3   33        No  Travel Frequently        1392  Research & Development   
4   27        No      Travel Rarely         591  Research & Development   

   Distance From Home      Education Education Field  EmployeeNumber  \
0                   1        College   Life Sciences               1   
1                   8  Below College   Life Sciences               2   
2                   2        College           Other               4   
3                   3         Master   Life Sciences               5   
4                   2  Below College         Medical               7   

  Environment Satisfaction  ... Performance Rating  Relationship Satisfaction  \
0                   Medium  ...    

**7. Outlier Detection (For Awareness in Visuals)**

In [19]:
def cap_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower_bound, upper_bound)
    return df

continuous_cols = [
    'Age', 'Monthly Income', 'Monthly Rate', 'Daily Rate', 'Hourly Rate',
    'Distance From Home', 'Percent Salary Hike', 'Total Working Years',
    'Years At Company', 'Years In Current Role', 'Years Since Last Promotion',
    'Years With Current Manager', 'Num Companies Worked'
]

for col in continuous_cols:
    df = cap_outliers(df, col)


In [20]:
df.to_csv("HR_Analytics_Clean.csv", index=False)